# USA House Prices Analysis & Prediction
The goal of this project is to explore and understand the factors influencing the real estate market in the United States and build machine learning models to accurately predict house prices.

During EDA, we aim to:

+ Identify key factors that influence house prices (e.g., location, size, number of rooms, year built).
+ Detect outliers and anomalies in the dataset.
+ Analyze the distribution of house prices and uncover trends.
+ Explore correlations between property features and prices.

> Source data: https://www.kaggle.com/datasets/farhankarim1/usa-house-prices

## Import libs

In [37]:
import pandas as pd

## Import raw data

In [38]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/Pytorch pet projects/ML - Projects/ML - USA HOUSE PRICES"
sys.path.append(os.path.join(project_path, "src"))

df = pd.read_csv("/content/drive/MyDrive/Pytorch pet projects/ML - Projects/ML - USA HOUSE PRICES/data/raw/USA_Housing.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Discover data

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
 6   Address                       5000 non-null   object 
dtypes: float64(6), object(1)
memory usage: 273.6+ KB


In [40]:
df.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,68583.108984,5.977222,6.987792,3.981330,36163.516039,1.232073e+06
std,10657.991214,0.991456,1.005833,1.234137,9925.650114,3.531176e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61480.562388,5.322283,6.299250,3.140000,29403.928702,9.975771e+05
50%,68804.286404,5.970429,7.002902,4.050000,36199.406689,1.232669e+06
75%,75783.338666,6.650808,7.665871,4.490000,42861.290769,1.471210e+06
max,107701.748378,9.519088,10.759588,6.500000,69621.713378,2.469066e+06


## Generating New Features for Better Predictions
+ Address features in this form is not the most usable.
Creating new features base on ```Address```

In [41]:
print(f"Ther is {len(df.Address.unique())} unique value in the Address feature")

Ther is 5000 unique value in the Address feature


In [42]:
df.Address.value_counts(normalize=True)

,proportion
Address,
"208 Michael Ferry Apt. 674\nLaurabury, NE 37010-5101",0.0002
"314 Christopher Square Apt. 404\nLake Ronaldville, SD 42025",0.0002
"21042 Wilson Islands Suite 238\nFischerchester, MP 42425-4129",0.0002
Unit 8831 Box 5748\nDPO AE 73012-7314,0.0002
"481 Kaitlin Mission Apt. 309\nJodystad, IA 16947",0.0002
...,...
"054 Carter Crescent Suite 674\nGlennport, WA 11140",0.0002
"8460 Kathleen Mission Apt. 482\nPort Amytown, KY 72016",0.0002
"3737 Hartman Rue\nReneestad, ID 69250-7718",0.0002


In [43]:
# Split the Address to Steet and to the City+State+Zip
df[["Street", "CityStateZip"]] = df["Address"].str.split("\n", expand=True)
df[["City", "StateZip"]] = df["CityStateZip"].str.split(", ", expand=True)
df[["State", "Zip"]] = df["StateZip"].str.split(" ", n=1, expand=True)

df.drop(columns=["Address", "CityStateZip", "StateZip"], axis=1, inplace=True)
df.head(5)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Street,City,State,Zip
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,208 Michael Ferry Apt. 674,Laurabury,NE,37010-5101
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,188 Johnson Views Suite 079,Lake Kathleen,CA,48958
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,9127 Elizabeth Stravenue,Danieltown,WI,06482-3489
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett,FPO AP 44820,None,None
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond,FPO AE 09386,None,None


## Handling Missing Data
+ There is 0 missing data in the dataframe, BUT there is a lot of value as ```None``` in the Address features.
